In [3]:
import sys
sys.path.append('/host/d/Github/')  # add the path to your own Example_UNet folder
import numpy as np
import nibabel as nb 
import pandas as pd
import os
import copy
import random
import Diffusion_denoising_thin_slice.simulation.ct_basic_PCD as ct
import Diffusion_denoising_thin_slice.functions_collection as ff
import ct_projector.projector.numpy.parallel as ct_para


main_path = '/host/d/Data/low_dose_CT/'  # change to your own data path

def split_angles_indexes(total_angles):
    arr = np.arange(total_angles)
   
    pairs = arr.reshape(-1, 2)  

    A = []
    B = []

    for pair in pairs:
        idx = random.randint(0, 1)  
        A.append(pair[idx])
        B.append(pair[1 - idx])

    A = np.array(A)
    B = np.array(B)
    return A,B

In [4]:
# define the patient list
patient_sheet = ff.find_all_target_files(['nii_imgs/*'],main_path)
print('Found {} patients'.format(len(patient_sheet)))

Found 10 patients


In [ ]:
split_angle_recon = True
all_angle_recon = True
for i in range(0, len(patient_sheet)):
    patient_id = os.path.basename(patient_sheet[i])

    print(patient_id)

    save_folder_case = os.path.join(main_path,'simulation_v2', patient_id)
    ff.make_folder([os.path.dirname(save_folder_case), save_folder_case])


    # img file
    img_file = os.path.join(patient_sheet[i],'img.nii.gz')
    print(img_file)
    # load img
    img_clean = nb.load(img_file).get_fdata().astype(np.float32)
    img_clean[img_clean < -1024] = -1024
    spacing = nb.load(img_file).header.get_zooms()[::-1]
    affine = nb.load(img_file).affine
    print('img shape, min, max: ', img_clean.shape, np.min(img_clean), np.max(img_clean))
    print('spacing: ', spacing)

    for noise_type in [ 'poisson']:
        # by vusialization we decided the following dose range
        # for MGH brain CT dataset
        # possion_hann_dose_range = [0.10,0.20]
        # gaussian_custom_dose_range = [0.15,0.25]

        # for mayo low does CT dataset
        dose_range = [0.30,0.40]#[0.15, 0.26] 
        
        for k in range(0,1):
            save_folder_k = os.path.join(save_folder_case, noise_type+'_random_'+str(k));ff.make_folder([save_folder_k])

            # if os.path.isfile(os.path.join(save_folder_k,'recon_odd.nii.gz')):
            #     print('already done, continue')
            #     continue

           
            if 1==3:#os.path.isfile(os.path.join(save_folder_k,'dose_factor.txt')):
            
                with open(os.path.join(save_folder_k,'dose_factor.txt'),'r') as f:
                    dose_factor = float(f.read())
                print('dose factor loaded: ', dose_factor)
            else:
                dose_factor = np.random.uniform(dose_range[0],dose_range[1] + 1e-8)
                # save into a txt file
                with open(os.path.join(save_folder_k,'dose_factor.txt'),'w') as f:
                    f.write('{:.4f}'.format(dose_factor))
            print('dose factor: ', dose_factor)


            # process img
            img0 = img_clean.copy()
            img0 = np.rollaxis(img0,-1,0)
      
            # define projectors
            geometry_file = '/host/d/Github/Diffusion_denoising_thin_slice/help_data/pcd_parallel_6x5_512.cfg'  # change to your own path
            projector = ct.define_forward_projector_pcd(img0,spacing, file_name = geometry_file)
            # FP
            # set angles
            angles = projector.get_angles()

            recon_all = np.zeros((img0.shape[1], img0.shape[2], img0.shape[0]), np.float32)
            recon_odd =  np.zeros((img0.shape[1], img0.shape[2], img0.shape[0]), np.float32)
            recon_even = np.zeros((img0.shape[1], img0.shape[2], img0.shape[0]), np.float32)
            # split angles
            if os.path.isfile(os.path.join(save_folder_k,'odd_even_indexes.npy')):
                odd_indexes = np.load(os.path.join(save_folder_k,'odd_even_indexes.npy'), allow_pickle=True).item()['odd']
                even_indexes = np.load(os.path.join(save_folder_k,'odd_even_indexes.npy'), allow_pickle=True).item()['even']
            else:
                odd_indexes, even_indexes = split_angles_indexes(len(angles))
                # save angles indexes for reference
                np.save(os.path.join(save_folder_k,'odd_even_indexes.npy'), {'odd':odd_indexes, 'even':even_indexes})

            for slice_n in range(0, img0.shape[0]):
                img_slice = img0[[slice_n],:,:].copy()
                img_slice = (img_slice[np.newaxis, ...] + 1000) / 1000 * 0.019 
                
                # forward proj
                prjs = ct_para.distance_driven_fp(projector, img_slice, angles)
                
                # add noise
                if noise_type[0:2] == 'po':
                    # add poisson noise
                    noise_of_prjs = ct.add_poisson_noise(prjs, N0=1000000, dose_factor = dose_factor) - prjs
                elif noise_type[0:2] == 'ga':
                    # add gaussian noise
                    noise_of_prjs = ct.add_gaussian_noise(prjs, N0=1000000, dose_factor = dose_factor) - prjs

                # recon
                if noise_type[0:2] == 'po':
                    # hann filter for noise projs
                    fnoise = ct_para.ramp_filter(projector, noise_of_prjs, 'hann')

                    # ramp filter for image
                    fprjs = ct_para.ramp_filter(projector, prjs, 'rl')

                    # recon all angles:
                    if all_angle_recon:
                        # bp for noise
                        recon_hann_noise = ct_para.distance_driven_bp(projector, fnoise, angles, True) 
                    
                        # bp for image
                        recon_hann_image = ct_para.distance_driven_bp(projector, fprjs, angles, True)
                    
                        # final recon
                        recon_hann = (recon_hann_noise +recon_hann_image)[0,0] / 0.019 * 1000 - 1000
                        
                        recon_all[:,:,slice_n] = recon_hann
                    
                    if split_angle_recon:
                        # split fnoise and fprjs to odd and even
                        fprjs_odd = fprjs[:, odd_indexes, ...]
                        fprjs_even = fprjs[:, even_indexes, ...]
                        fnoise_odd = fnoise[:, odd_indexes, ...]
                        fnoise_even = fnoise[:, even_indexes, ...]
                
                        # bp for image
                        recon_hann_image_odd = ct_para.distance_driven_bp(projector, fprjs_odd, angles[odd_indexes], True)
                        recon_hann_image_even = ct_para.distance_driven_bp(projector, fprjs_even, angles[even_indexes], True)

                        # bp for noise
                        recon_hann_noise_odd = ct_para.distance_driven_bp(projector, fnoise_odd, angles[odd_indexes], True)
                        recon_hann_noise_even = ct_para.distance_driven_bp(projector, fnoise_even, angles[even_indexes], True)

                        # final recon
                        recon_hann_odd = (recon_hann_image_odd + recon_hann_noise_odd) *2 / 0.019 * 1000 - 1000
                        recon_hann_even = (recon_hann_image_even + recon_hann_noise_even) *2 / 0.019 * 1000 - 1000
                        recon_odd[:,:,slice_n] = recon_hann_odd
                        recon_even[:,:,slice_n] = recon_hann_even
                        
                elif noise_type[0:2] == 'ga':

                    # custom filter
                    custom_filter_file = '/host/d/Github/Diffusion_denoising_thin_slice/help_data/softTissueKernel_65'  # change to your own path
                    custom_additional_filter = ct.get_additional_filter_to_rl(custom_filter_file, projector.nu, projector.du, projector.nview)

                    # ramp filter for image
                    fprjs, projector_new = ct.interleave_filter_and_recon(projector, prjs, custom_additional_filter, angles, get_recon=False, ramp_filter=True)
                    
                    # soft tissue kernel for noise
                    fnoise, _ = ct.interleave_filter_and_recon(projector, noise_of_prjs, custom_additional_filter, angles, get_recon=False , ramp_filter=False)
                    
                    if all_angle_recon:
                        # bp for image
                        recon_custom_image = ct_para.pixel_driven_bp(projector_new, fprjs, angles)
                        # bp for noise
                        recon_custom_noise = ct_para.pixel_driven_bp(projector_new, fnoise, angles)
                        # final recon
                        recon_custom = (recon_custom_noise + recon_custom_image)/ 0.019 * 1000 - 1000

                        recon_all[:,:,slice_n] = recon_custom
                    
                    if split_angle_recon:

                        # split fprjs and fnoise to odd and even
                        fprjs_odd = fprjs[:, odd_indexes, ...]
                        fprjs_even = fprjs[:, even_indexes, ...] 
                        fnoise_odd = fnoise[:, odd_indexes, ...]
                        fnoise_even = fnoise[:, even_indexes, ...]
                        
                        # # bp for image
                        recon_custom_image_odd = ct_para.pixel_driven_bp(projector_new, fprjs_odd, angles[odd_indexes])
                        recon_custom_image_even = ct_para.pixel_driven_bp(projector_new, fprjs_even, angles[even_indexes])

                        # # bp for noise
                        recon_custom_noise_odd = ct_para.pixel_driven_bp(projector_new, fnoise_odd, angles[odd_indexes])
                        recon_custom_noise_even = ct_para.pixel_driven_bp(projector_new, fnoise_even, angles[even_indexes])

                        # final recon
                        recon_custom_odd = (recon_custom_image_odd + recon_custom_noise_odd) *2  / 0.019 * 1000 - 1000
                        recon_custom_even = ( recon_custom_image_even + recon_custom_noise_even) *2 / 0.019 * 1000 - 1000

                        recon_odd[:,:,slice_n] = recon_custom_odd
                        recon_even[:,:,slice_n] = recon_custom_even
                        
            # save recon
            # nb.save(nb.Nifti1Image(img_clean, affine), os.path.join(save_folder_case,'img_clean.nii.gz'))
            if all_angle_recon:
                nb.save(nb.Nifti1Image(recon_all, affine), os.path.join(save_folder_k,'recon_all.nii.gz'))
                diff = recon_all - img_clean
                nb.save(nb.Nifti1Image(diff, affine), os.path.join(save_folder_k,'diff_clean_all.nii.gz'))
            if split_angle_recon:
                nb.save(nb.Nifti1Image(recon_odd, affine), os.path.join(save_folder_k,'recon_odd.nii.gz'))
                nb.save(nb.Nifti1Image(recon_even, affine), os.path.join(save_folder_k,'recon_even.nii.gz'))
                diff_between = recon_odd - recon_even
                nb.save(nb.Nifti1Image(diff_between, affine), os.path.join(save_folder_k,'diff_odd_even.nii.gz'))
            

L067
/host/d/Data/low_dose_CT/nii_imgs/L067/img.nii.gz
img shape, min, max:  (512, 512, 224) -1024.0 2123.0
spacing:  (2.0, 0.6640625, 0.6640625)
dose factor:  0.3542905550740003
